<a href="https://colab.research.google.com/github/MisalChugh/Pyspark_Notebooks/blob/Notebooks/Snapshot_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark==3.2.1

import pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 kB 18.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853629 sha256=0db1623e5b193c7abd07e8c27b51e57e232296c5535a6eac2d393040d0766bf6
  Stored in directory: /root/.cache/pip/wheels/15/97/bd/52908574a60b5f8e3dc4dc5a0b5be8a59ac20986ee51c2611b
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, min, max, current_date, expr, sum, lit, when
from pyspark.sql.types import DateType
from pyspark.sql.window import Window

spark = SparkSession.builder\
    .appName("InventorySnapshot")\
        .getOrCreate()

In [ ]:
print("Reading Source file -->")
inventory_df = spark.read.csv(r"/content/drive/MyDrive/Snapshot_Assignment/SnapShot_input.csv", header=True, inferSchema=True)
inventory_df.show()
print("Count -->", inventory_df.count())
inventory_df.printSchema()

Reading Source file -->
+----------+-----------+--------+-------+
|      Date|Branch_Code|lot_code|net_qty|
+----------+-----------+--------+-------+
|2020-03-31|       5013| 4499295|      6|
|2020-06-15|       5013| 4499295|     -2|
|2020-10-29|       5013| 4743770|     24|
|2020-10-29|       5013| 4743771|     18|
|2020-10-29|       5013| 4743769|     24|
|2020-10-29|       5013| 4743772|     24|
|2020-10-29|       5013| 4743773|     24|
|2020-10-30|       5013| 4743773|    -10|
|2020-10-30|       5013| 4743771|    -18|
|2020-11-02|       5013| 4743773|     -6|
|2020-11-07|       5013| 4743773|     -6|
|2020-11-08|       5013| 4743769|     -3|
|2020-11-11|       5013| 4743773|     -2|
|2020-11-11|       5013| 4743773|     18|
|2020-12-11|       5013| 4743772|     -4|
|2020-12-11|       5013| 4743770|     -6|
|2020-11-13|       5013| 4743769|     -2|
|2020-11-14|       5013| 4743772|     -2|
|2020-11-15|       5013| 4743772|     -1|
|2020-11-19|       5013| 4743773|     -6|
+---------

In [ ]:
print("Cast Date column -->")
inventory_df = inventory_df.withColumn("Date", col("Date").cast(DateType()))
inventory_df.printSchema()
print("Count -->", inventory_df.count())
inventory_df.show()

Cast Date column -->
root
 |-- Date: date (nullable = true)
 |-- Branch_Code: integer (nullable = true)
 |-- lot_code: integer (nullable = true)
 |-- net_qty: integer (nullable = true)

Count --> 43
+----------+-----------+--------+-------+
|      Date|Branch_Code|lot_code|net_qty|
+----------+-----------+--------+-------+
|2020-03-31|       5013| 4499295|      6|
|2020-06-15|       5013| 4499295|     -2|
|2020-10-29|       5013| 4743770|     24|
|2020-10-29|       5013| 4743771|     18|
|2020-10-29|       5013| 4743769|     24|
|2020-10-29|       5013| 4743772|     24|
|2020-10-29|       5013| 4743773|     24|
|2020-10-30|       5013| 4743773|    -10|
|2020-10-30|       5013| 4743771|    -18|
|2020-11-02|       5013| 4743773|     -6|
|2020-11-07|       5013| 4743773|     -6|
|2020-11-08|       5013| 4743769|     -3|
|2020-11-11|       5013| 4743773|     -2|
|2020-11-11|       5013| 4743773|     18|
|2020-12-11|       5013| 4743772|     -4|
|2020-12-11|       5013| 4743770|     -6|
|20

In [ ]:
print("Calculate Min_date -->")
date_df = inventory_df.select(min("Date").cast(DateType()).alias("Min_date"))
date_df.show()
date_df.printSchema()

Calculate Min_date -->
+----------+
|  Min_date|
+----------+
|2020-03-31|
+----------+

root
 |-- Min_date: date (nullable = true)



In [ ]:
print("Add CurrentDate column -->")
date_df = date_df.withColumn("CurrentDate", current_date())
date_df.show()
date_df.printSchema()

Add CurrentDate column -->
+----------+-----------+
|  Min_date|CurrentDate|
+----------+-----------+
|2020-03-31| 2023-07-02|
+----------+-----------+

root
 |-- Min_date: date (nullable = true)
 |-- CurrentDate: date (nullable = false)



In [ ]:
print("Calculate Date range -->")
date_df = date_df.withColumn("Date_x", explode(expr("sequence(Min_date, CurrentDate, interval 1 day)")))
print("Count -->", date_df.count())
date_df.show()

Calculate Date range -->
Count --> 1189
+----------+-----------+----------+
|  Min_date|CurrentDate|    Date_x|
+----------+-----------+----------+
|2020-03-31| 2023-07-02|2020-03-31|
|2020-03-31| 2023-07-02|2020-04-01|
|2020-03-31| 2023-07-02|2020-04-02|
|2020-03-31| 2023-07-02|2020-04-03|
|2020-03-31| 2023-07-02|2020-04-04|
|2020-03-31| 2023-07-02|2020-04-05|
|2020-03-31| 2023-07-02|2020-04-06|
|2020-03-31| 2023-07-02|2020-04-07|
|2020-03-31| 2023-07-02|2020-04-08|
|2020-03-31| 2023-07-02|2020-04-09|
|2020-03-31| 2023-07-02|2020-04-10|
|2020-03-31| 2023-07-02|2020-04-11|
|2020-03-31| 2023-07-02|2020-04-12|
|2020-03-31| 2023-07-02|2020-04-13|
|2020-03-31| 2023-07-02|2020-04-14|
|2020-03-31| 2023-07-02|2020-04-15|
|2020-03-31| 2023-07-02|2020-04-16|
|2020-03-31| 2023-07-02|2020-04-17|
|2020-03-31| 2023-07-02|2020-04-18|
|2020-03-31| 2023-07-02|2020-04-19|
+----------+-----------+----------+
only showing top 20 rows



In [ ]:
print("Drop columns -->")
date_df = date_df.drop("Min_date", "CurrentDate")
print("Count -->", date_df.count())
date_df.show()

Drop columns -->
Count --> 1189
+----------+
|    Date_x|
+----------+
|2020-03-31|
|2020-04-01|
|2020-04-02|
|2020-04-03|
|2020-04-04|
|2020-04-05|
|2020-04-06|
|2020-04-07|
|2020-04-08|
|2020-04-09|
|2020-04-10|
|2020-04-11|
|2020-04-12|
|2020-04-13|
|2020-04-14|
|2020-04-15|
|2020-04-16|
|2020-04-17|
|2020-04-18|
|2020-04-19|
+----------+
only showing top 20 rows



In [ ]:
print("Add column for joining -->")
date_df = date_df.withColumn("value", lit(1))
print("Count -->", date_df.count())
date_df.show()

Add column for joining -->
Count --> 1189
+----------+-----+
|    Date_x|value|
+----------+-----+
|2020-03-31|    1|
|2020-04-01|    1|
|2020-04-02|    1|
|2020-04-03|    1|
|2020-04-04|    1|
|2020-04-05|    1|
|2020-04-06|    1|
|2020-04-07|    1|
|2020-04-08|    1|
|2020-04-09|    1|
|2020-04-10|    1|
|2020-04-11|    1|
|2020-04-12|    1|
|2020-04-13|    1|
|2020-04-14|    1|
|2020-04-15|    1|
|2020-04-16|    1|
|2020-04-17|    1|
|2020-04-18|    1|
|2020-04-19|    1|
+----------+-----+
only showing top 20 rows



In [ ]:
print("Group by --> ")
group_df = inventory_df.groupBy("Branch_Code", "lot_code").agg(min("Date").alias("min_date"),
                                                          max("Date").alias("max_date"),
                                                          sum("net_qty").alias("Sum")
                                                          )
print("Count -->", group_df.count())
group_df.show()

Group by --> 
Count --> 10
+-----------+--------+----------+----------+---+
|Branch_Code|lot_code|  min_date|  max_date|Sum|
+-----------+--------+----------+----------+---+
|       5013| 4743769|2020-10-29|2020-12-31|  0|
|       5013| 4743770|2020-10-29|2021-01-16| 16|
|       5013| 4826751|2020-12-23|2020-12-31| 10|
|       5013| 4854774|2021-01-05|2021-01-05| 18|
|       5013| 4854771|2021-01-05|2021-01-05| 12|
|       5013| 4804612|2020-12-08|2021-01-20|  6|
|       5013| 4743773|2020-10-29|2020-12-17|  0|
|       5013| 4743771|2020-10-29|2020-10-30|  0|
|       5013| 4499295|2020-03-31|2020-12-01|  3|
|       5013| 4743772|2020-10-29|2021-01-07|  4|
+-----------+--------+----------+----------+---+



In [ ]:
print("Add column for joining -->")
group_df = group_df.withColumn("value", lit(1))
print("Count -->", group_df.count())
group_df.show()

Add column for joining -->
Count --> 10
+-----------+--------+----------+----------+---+-----+
|Branch_Code|lot_code|  min_date|  max_date|Sum|value|
+-----------+--------+----------+----------+---+-----+
|       5013| 4743769|2020-10-29|2020-12-31|  0|    1|
|       5013| 4743770|2020-10-29|2021-01-16| 16|    1|
|       5013| 4826751|2020-12-23|2020-12-31| 10|    1|
|       5013| 4854774|2021-01-05|2021-01-05| 18|    1|
|       5013| 4854771|2021-01-05|2021-01-05| 12|    1|
|       5013| 4804612|2020-12-08|2021-01-20|  6|    1|
|       5013| 4743773|2020-10-29|2020-12-17|  0|    1|
|       5013| 4743771|2020-10-29|2020-10-30|  0|    1|
|       5013| 4499295|2020-03-31|2020-12-01|  3|    1|
|       5013| 4743772|2020-10-29|2021-01-07|  4|    1|
+-----------+--------+----------+----------+---+-----+



In [ ]:
print("Join date_df and group_df -->")
join_df = date_df.join(group_df, on="value", how="outer")
print("Count -->", join_df.count())
join_df.show()

Join date_df and group_df -->
Count --> 11890
+-----+----------+-----------+--------+----------+----------+---+
|value|    Date_x|Branch_Code|lot_code|  min_date|  max_date|Sum|
+-----+----------+-----------+--------+----------+----------+---+
|    1|2020-03-31|       5013| 4743769|2020-10-29|2020-12-31|  0|
|    1|2020-04-01|       5013| 4743769|2020-10-29|2020-12-31|  0|
|    1|2020-04-02|       5013| 4743769|2020-10-29|2020-12-31|  0|
|    1|2020-04-03|       5013| 4743769|2020-10-29|2020-12-31|  0|
|    1|2020-04-04|       5013| 4743769|2020-10-29|2020-12-31|  0|
|    1|2020-04-05|       5013| 4743769|2020-10-29|2020-12-31|  0|
|    1|2020-04-06|       5013| 4743769|2020-10-29|2020-12-31|  0|
|    1|2020-04-07|       5013| 4743769|2020-10-29|2020-12-31|  0|
|    1|2020-04-08|       5013| 4743769|2020-10-29|2020-12-31|  0|
|    1|2020-04-09|       5013| 4743769|2020-10-29|2020-12-31|  0|
|    1|2020-04-10|       5013| 4743769|2020-10-29|2020-12-31|  0|
|    1|2020-04-11|       5013|

In [ ]:
print("Filter Date -->")
join_df = join_df.filter(join_df.Date_x >= join_df.min_date)
print("Count -->", join_df.count())
join_df.show()

Filter Date -->
Count --> 9751
+-----+----------+-----------+--------+----------+----------+---+
|value|    Date_x|Branch_Code|lot_code|  min_date|  max_date|Sum|
+-----+----------+-----------+--------+----------+----------+---+
|    1|2020-10-29|       5013| 4743769|2020-10-29|2020-12-31|  0|
|    1|2020-10-30|       5013| 4743769|2020-10-29|2020-12-31|  0|
|    1|2020-10-31|       5013| 4743769|2020-10-29|2020-12-31|  0|
|    1|2020-11-01|       5013| 4743769|2020-10-29|2020-12-31|  0|
|    1|2020-11-02|       5013| 4743769|2020-10-29|2020-12-31|  0|
|    1|2020-11-03|       5013| 4743769|2020-10-29|2020-12-31|  0|
|    1|2020-11-04|       5013| 4743769|2020-10-29|2020-12-31|  0|
|    1|2020-11-05|       5013| 4743769|2020-10-29|2020-12-31|  0|
|    1|2020-11-06|       5013| 4743769|2020-10-29|2020-12-31|  0|
|    1|2020-11-07|       5013| 4743769|2020-10-29|2020-12-31|  0|
|    1|2020-11-08|       5013| 4743769|2020-10-29|2020-12-31|  0|
|    1|2020-11-09|       5013| 4743769|2020-1

In [ ]:
print("Add flag column -->")
join_df = join_df.withColumn("flag", when((join_df.Date_x > join_df.max_date) &
                                          (join_df.Sum == 0), lit(0)).otherwise(lit(1)))
print("Count -->", join_df.count())
join_df.show()

Add flag column -->
Count --> 9751
+-----+----------+-----------+--------+----------+----------+---+----+
|value|    Date_x|Branch_Code|lot_code|  min_date|  max_date|Sum|flag|
+-----+----------+-----------+--------+----------+----------+---+----+
|    1|2020-10-29|       5013| 4743769|2020-10-29|2020-12-31|  0|   1|
|    1|2020-10-30|       5013| 4743769|2020-10-29|2020-12-31|  0|   1|
|    1|2020-10-31|       5013| 4743769|2020-10-29|2020-12-31|  0|   1|
|    1|2020-11-01|       5013| 4743769|2020-10-29|2020-12-31|  0|   1|
|    1|2020-11-02|       5013| 4743769|2020-10-29|2020-12-31|  0|   1|
|    1|2020-11-03|       5013| 4743769|2020-10-29|2020-12-31|  0|   1|
|    1|2020-11-04|       5013| 4743769|2020-10-29|2020-12-31|  0|   1|
|    1|2020-11-05|       5013| 4743769|2020-10-29|2020-12-31|  0|   1|
|    1|2020-11-06|       5013| 4743769|2020-10-29|2020-12-31|  0|   1|
|    1|2020-11-07|       5013| 4743769|2020-10-29|2020-12-31|  0|   1|
|    1|2020-11-08|       5013| 4743769|202

In [ ]:
print("Filter where flag == 1 -->")
join_df = join_df.filter(join_df.flag == 1)
print("Count -->", join_df.count())
join_df.show()

Filter where flag == 1 -->
Count --> 6936
+-----+----------+-----------+--------+----------+----------+---+----+
|value|    Date_x|Branch_Code|lot_code|  min_date|  max_date|Sum|flag|
+-----+----------+-----------+--------+----------+----------+---+----+
|    1|2020-10-29|       5013| 4743769|2020-10-29|2020-12-31|  0|   1|
|    1|2020-10-30|       5013| 4743769|2020-10-29|2020-12-31|  0|   1|
|    1|2020-10-31|       5013| 4743769|2020-10-29|2020-12-31|  0|   1|
|    1|2020-11-01|       5013| 4743769|2020-10-29|2020-12-31|  0|   1|
|    1|2020-11-02|       5013| 4743769|2020-10-29|2020-12-31|  0|   1|
|    1|2020-11-03|       5013| 4743769|2020-10-29|2020-12-31|  0|   1|
|    1|2020-11-04|       5013| 4743769|2020-10-29|2020-12-31|  0|   1|
|    1|2020-11-05|       5013| 4743769|2020-10-29|2020-12-31|  0|   1|
|    1|2020-11-06|       5013| 4743769|2020-10-29|2020-12-31|  0|   1|
|    1|2020-11-07|       5013| 4743769|2020-10-29|2020-12-31|  0|   1|
|    1|2020-11-08|       5013| 4743

In [ ]:
print("Drop Branch_Code column -->")
inventory_df = inventory_df.drop("Branch_Code")
print("Count -->", inventory_df.count())
inventory_df.show()

Drop Branch_Code column -->
Count --> 43
+----------+--------+-------+
|      Date|lot_code|net_qty|
+----------+--------+-------+
|2020-03-31| 4499295|      6|
|2020-06-15| 4499295|     -2|
|2020-10-29| 4743770|     24|
|2020-10-29| 4743771|     18|
|2020-10-29| 4743769|     24|
|2020-10-29| 4743772|     24|
|2020-10-29| 4743773|     24|
|2020-10-30| 4743773|    -10|
|2020-10-30| 4743771|    -18|
|2020-11-02| 4743773|     -6|
|2020-11-07| 4743773|     -6|
|2020-11-08| 4743769|     -3|
|2020-11-11| 4743773|     -2|
|2020-11-11| 4743773|     18|
|2020-12-11| 4743772|     -4|
|2020-12-11| 4743770|     -6|
|2020-11-13| 4743769|     -2|
|2020-11-14| 4743772|     -2|
|2020-11-15| 4743772|     -1|
|2020-11-19| 4743773|     -6|
+----------+--------+-------+
only showing top 20 rows



In [ ]:
print("Join join_df and inventory_df -->")
main_join_df = join_df.join(inventory_df, on='lot_code', how='outer')
print("Count -->", main_join_df.count())
main_join_df.show()

Join join_df and inventory_df -->
Count --> 25462
+--------+-----+----------+-----------+----------+----------+---+----+----------+-------+
|lot_code|value|    Date_x|Branch_Code|  min_date|  max_date|Sum|flag|      Date|net_qty|
+--------+-----+----------+-----------+----------+----------+---+----+----------+-------+
| 4499295|    1|2020-03-31|       5013|2020-03-31|2020-12-01|  3|   1|2020-03-31|      6|
| 4499295|    1|2020-03-31|       5013|2020-03-31|2020-12-01|  3|   1|2020-06-15|     -2|
| 4499295|    1|2020-03-31|       5013|2020-03-31|2020-12-01|  3|   1|2020-12-01|     -1|
| 4499295|    1|2020-04-01|       5013|2020-03-31|2020-12-01|  3|   1|2020-03-31|      6|
| 4499295|    1|2020-04-01|       5013|2020-03-31|2020-12-01|  3|   1|2020-06-15|     -2|
| 4499295|    1|2020-04-01|       5013|2020-03-31|2020-12-01|  3|   1|2020-12-01|     -1|
| 4499295|    1|2020-04-02|       5013|2020-03-31|2020-12-01|  3|   1|2020-03-31|      6|
| 4499295|    1|2020-04-02|       5013|2020-03-31|

In [ ]:
print("Drop columns -->")
main_join_df = main_join_df.drop("value")
print("Count -->", main_join_df.count())
main_join_df.show()

Drop columns -->
Count --> 25462
+--------+----------+-----------+----------+----------+---+----+----------+-------+
|lot_code|    Date_x|Branch_Code|  min_date|  max_date|Sum|flag|      Date|net_qty|
+--------+----------+-----------+----------+----------+---+----+----------+-------+
| 4499295|2020-03-31|       5013|2020-03-31|2020-12-01|  3|   1|2020-03-31|      6|
| 4499295|2020-03-31|       5013|2020-03-31|2020-12-01|  3|   1|2020-06-15|     -2|
| 4499295|2020-03-31|       5013|2020-03-31|2020-12-01|  3|   1|2020-12-01|     -1|
| 4499295|2020-04-01|       5013|2020-03-31|2020-12-01|  3|   1|2020-03-31|      6|
| 4499295|2020-04-01|       5013|2020-03-31|2020-12-01|  3|   1|2020-06-15|     -2|
| 4499295|2020-04-01|       5013|2020-03-31|2020-12-01|  3|   1|2020-12-01|     -1|
| 4499295|2020-04-02|       5013|2020-03-31|2020-12-01|  3|   1|2020-03-31|      6|
| 4499295|2020-04-02|       5013|2020-03-31|2020-12-01|  3|   1|2020-06-15|     -2|
| 4499295|2020-04-02|       5013|2020-03-31

In [ ]:
print("Rename Date column -->")
main_join_df = main_join_df.withColumnRenamed("Date", "Date_y")
print("Count -->", main_join_df.count())
main_join_df.show()

Rename Date column -->
Count --> 25462
+--------+----------+-----------+----------+----------+---+----+----------+-------+
|lot_code|    Date_x|Branch_Code|  min_date|  max_date|Sum|flag|    Date_y|net_qty|
+--------+----------+-----------+----------+----------+---+----+----------+-------+
| 4499295|2020-03-31|       5013|2020-03-31|2020-12-01|  3|   1|2020-03-31|      6|
| 4499295|2020-03-31|       5013|2020-03-31|2020-12-01|  3|   1|2020-06-15|     -2|
| 4499295|2020-03-31|       5013|2020-03-31|2020-12-01|  3|   1|2020-12-01|     -1|
| 4499295|2020-04-01|       5013|2020-03-31|2020-12-01|  3|   1|2020-03-31|      6|
| 4499295|2020-04-01|       5013|2020-03-31|2020-12-01|  3|   1|2020-06-15|     -2|
| 4499295|2020-04-01|       5013|2020-03-31|2020-12-01|  3|   1|2020-12-01|     -1|
| 4499295|2020-04-02|       5013|2020-03-31|2020-12-01|  3|   1|2020-03-31|      6|
| 4499295|2020-04-02|       5013|2020-03-31|2020-12-01|  3|   1|2020-06-15|     -2|
| 4499295|2020-04-02|       5013|2020

In [ ]:
print("Add flag2 column -->")
main_join_df = main_join_df.withColumn("flag2", when(main_join_df.Date_x == main_join_df.Date_y, lit(main_join_df.net_qty)).otherwise(lit(0)))
print("Count -->", main_join_df.count())
main_join_df.show()

Add flag2 column -->
Count --> 25462
+--------+----------+-----------+----------+----------+---+----+----------+-------+-----+
|lot_code|    Date_x|Branch_Code|  min_date|  max_date|Sum|flag|    Date_y|net_qty|flag2|
+--------+----------+-----------+----------+----------+---+----+----------+-------+-----+
| 4499295|2020-03-31|       5013|2020-03-31|2020-12-01|  3|   1|2020-03-31|      6|    6|
| 4499295|2020-03-31|       5013|2020-03-31|2020-12-01|  3|   1|2020-06-15|     -2|    0|
| 4499295|2020-03-31|       5013|2020-03-31|2020-12-01|  3|   1|2020-12-01|     -1|    0|
| 4499295|2020-04-01|       5013|2020-03-31|2020-12-01|  3|   1|2020-03-31|      6|    0|
| 4499295|2020-04-01|       5013|2020-03-31|2020-12-01|  3|   1|2020-06-15|     -2|    0|
| 4499295|2020-04-01|       5013|2020-03-31|2020-12-01|  3|   1|2020-12-01|     -1|    0|
| 4499295|2020-04-02|       5013|2020-03-31|2020-12-01|  3|   1|2020-03-31|      6|    0|
| 4499295|2020-04-02|       5013|2020-03-31|2020-12-01|  3|   1

In [ ]:
print("Rename columns -->")
main_join_df = main_join_df.withColumnRenamed("Date_x", "Date").withColumnRenamed("cumsum", "net_qty")
print("Count -->", main_join_df.count())
main_join_df.show()

Rename columns -->
Count --> 25462
+--------+----------+-----------+----------+----------+---+----+----------+-------+
|lot_code|      Date|Branch_Code|  min_date|  max_date|Sum|flag|      Date|net_qty|
+--------+----------+-----------+----------+----------+---+----+----------+-------+
| 4499295|2020-03-31|       5013|2020-03-31|2020-12-01|  3|   1|2020-03-31|      6|
| 4499295|2020-03-31|       5013|2020-03-31|2020-12-01|  3|   1|2020-06-15|     -2|
| 4499295|2020-03-31|       5013|2020-03-31|2020-12-01|  3|   1|2020-12-01|     -1|
| 4499295|2020-04-01|       5013|2020-03-31|2020-12-01|  3|   1|2020-03-31|      6|
| 4499295|2020-04-01|       5013|2020-03-31|2020-12-01|  3|   1|2020-06-15|     -2|
| 4499295|2020-04-01|       5013|2020-03-31|2020-12-01|  3|   1|2020-12-01|     -1|
| 4499295|2020-04-02|       5013|2020-03-31|2020-12-01|  3|   1|2020-03-31|      6|
| 4499295|2020-04-02|       5013|2020-03-31|2020-12-01|  3|   1|2020-06-15|     -2|
| 4499295|2020-04-02|       5013|2020-03-

In [ ]:
print("Calculate cumsum column -->")
windowSpec = Window.partitionBy("Branch_Code", "lot_code").orderBy("Date_x")
main_join_df = main_join_df.withColumn('cumsum', sum("flag2").over(windowSpec))
print("Count -->", main_join_df.count())
main_join_df.show()

Calculate cumsum column -->
Count --> 25462
+--------+----------+-----------+----------+----------+---+----+----------+-------+-----+------+
|lot_code|    Date_x|Branch_Code|  min_date|  max_date|Sum|flag|    Date_y|net_qty|flag2|cumsum|
+--------+----------+-----------+----------+----------+---+----+----------+-------+-----+------+
| 4499295|2020-03-31|       5013|2020-03-31|2020-12-01|  3|   1|2020-03-31|      6|    6|     6|
| 4499295|2020-03-31|       5013|2020-03-31|2020-12-01|  3|   1|2020-06-15|     -2|    0|     6|
| 4499295|2020-03-31|       5013|2020-03-31|2020-12-01|  3|   1|2020-12-01|     -1|    0|     6|
| 4499295|2020-04-01|       5013|2020-03-31|2020-12-01|  3|   1|2020-03-31|      6|    0|     6|
| 4499295|2020-04-01|       5013|2020-03-31|2020-12-01|  3|   1|2020-06-15|     -2|    0|     6|
| 4499295|2020-04-01|       5013|2020-03-31|2020-12-01|  3|   1|2020-12-01|     -1|    0|     6|
| 4499295|2020-04-02|       5013|2020-03-31|2020-12-01|  3|   1|2020-03-31|      6|

In [ ]:
print("Add flag3 column -->")
main_join_df = main_join_df.withColumn("flag3",when((main_join_df.max_date < main_join_df.Date_y)&(main_join_df.Sum==0),lit(0)).otherwise(lit(1)))
print("Count -->", main_join_df.count())
main_join_df.show()

Add flag3 column -->
Count --> 25462
+--------+----------+-----------+----------+----------+---+----+----------+-------+-----+------+-----+
|lot_code|    Date_x|Branch_Code|  min_date|  max_date|Sum|flag|    Date_y|net_qty|flag2|cumsum|flag3|
+--------+----------+-----------+----------+----------+---+----+----------+-------+-----+------+-----+
| 4499295|2020-03-31|       5013|2020-03-31|2020-12-01|  3|   1|2020-03-31|      6|    6|     6|    1|
| 4499295|2020-03-31|       5013|2020-03-31|2020-12-01|  3|   1|2020-06-15|     -2|    0|     6|    1|
| 4499295|2020-03-31|       5013|2020-03-31|2020-12-01|  3|   1|2020-12-01|     -1|    0|     6|    1|
| 4499295|2020-04-01|       5013|2020-03-31|2020-12-01|  3|   1|2020-03-31|      6|    0|     6|    1|
| 4499295|2020-04-01|       5013|2020-03-31|2020-12-01|  3|   1|2020-06-15|     -2|    0|     6|    1|
| 4499295|2020-04-01|       5013|2020-03-31|2020-12-01|  3|   1|2020-12-01|     -1|    0|     6|    1|
| 4499295|2020-04-02|       5013|202

In [ ]:
print("Drop and Renaming columns -->")
main_join_df = main_join_df.drop("min_date", "max_date", "Sum", "flag", "Date_y", "flag2", "flag3", "net_qty")
main_join_df = main_join_df.withColumnRenamed("Date_x", "Date").withColumnRenamed("cumsum", "net_qty")
print("Count -->", main_join_df.count())
main_join_df.show()

Drop and Renaming columns -->
Count --> 25462
+--------+----------+-----------+-------+
|lot_code|      Date|Branch_Code|net_qty|
+--------+----------+-----------+-------+
| 4499295|2020-03-31|       5013|      6|
| 4499295|2020-03-31|       5013|      6|
| 4499295|2020-03-31|       5013|      6|
| 4499295|2020-04-01|       5013|      6|
| 4499295|2020-04-01|       5013|      6|
| 4499295|2020-04-01|       5013|      6|
| 4499295|2020-04-02|       5013|      6|
| 4499295|2020-04-02|       5013|      6|
| 4499295|2020-04-02|       5013|      6|
| 4499295|2020-04-03|       5013|      6|
| 4499295|2020-04-03|       5013|      6|
| 4499295|2020-04-03|       5013|      6|
| 4499295|2020-04-04|       5013|      6|
| 4499295|2020-04-04|       5013|      6|
| 4499295|2020-04-04|       5013|      6|
| 4499295|2020-04-05|       5013|      6|
| 4499295|2020-04-05|       5013|      6|
| 4499295|2020-04-05|       5013|      6|
| 4499295|2020-04-06|       5013|      6|
| 4499295|2020-04-06|       50

In [ ]:
print("Remove Duplicate Rows -->")
main_join_df = main_join_df.distinct()
print("Count -->", main_join_df.count())
main_join_df.show()

Remove Duplicate Rows -->
Count --> 6936
+--------+----------+-----------+-------+
|lot_code|      Date|Branch_Code|net_qty|
+--------+----------+-----------+-------+
| 4499295|2020-03-31|       5013|      6|
| 4499295|2020-04-01|       5013|      6|
| 4499295|2020-04-02|       5013|      6|
| 4499295|2020-04-03|       5013|      6|
| 4499295|2020-04-04|       5013|      6|
| 4499295|2020-04-05|       5013|      6|
| 4499295|2020-04-06|       5013|      6|
| 4499295|2020-04-07|       5013|      6|
| 4499295|2020-04-08|       5013|      6|
| 4499295|2020-04-09|       5013|      6|
| 4499295|2020-04-10|       5013|      6|
| 4499295|2020-04-11|       5013|      6|
| 4499295|2020-04-12|       5013|      6|
| 4499295|2020-04-13|       5013|      6|
| 4499295|2020-04-14|       5013|      6|
| 4499295|2020-04-15|       5013|      6|
| 4499295|2020-04-16|       5013|      6|
| 4499295|2020-04-17|       5013|      6|
| 4499295|2020-04-18|       5013|      6|
| 4499295|2020-04-19|       5013|  

In [ ]:
print("Writing the CSV file -->")
output_path = "/content/drive/MyDrive/Snapshot_Assignment/output"
main_join_df.write.csv(output_path, header=True)
print("Output written to:", output_path)

Writing the CSV file -->
Output written to: /content/drive/MyDrive/Snapshot_Assignment/output
